# Load, process and Plot CSV files

# 1. Notebook Preparation

## 1.1 Import libraries

In [1]:
#General
import os
import glob
import numpy as np
import pandas as pd
import dtale

#Plotly libraries
import plotly.graph_objs as go
import plotly
import cufflinks as cf
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#Plotly Initialization for Jupyter notebook
init_notebook_mode(connected=True)
cf.go_offline()

## 1.2 Functions

In [2]:
os.chdir(r'C:\Users\m0081459\Documents\Jupyter') #Copy directory where other ipynb are located
%run -i Functions.ipynb
%run -i Columns.ipynb

# 2. Load files and set folder location

## 2.1 Request and retrieve logs

In [36]:
#Folder directory
print("Files Folder")
InDirectory = input(prompt=">>>")
os.chdir(InDirectory)

#Explore directory and select all files with the given format
import_extension = 'csv'
files = [i for i in sorted(glob.glob('*.{}'.format(import_extension)), key=os.path.getmtime)] #Change to *.**{} to search in subfolders

Files Folder


KeyboardInterrupt: Interrupted by user

### 2.1.1 Check files uploaded

In [4]:
try:
    print('Files selected:')
    for count,file in enumerate(files):
        print(count+1,'-',get_file_size(file),'MB -',file)
except:
    print('Error','The list of files to import is empty. Click on the Upload File button and select a log file')

Files selected:
1 - 40.54 MB - Sample 170842 Date 11_19_2021 - Time 10_56_55 AM.csv


## 2.2 Choose the Ouput Directory

In [5]:
print("Output Folder")
OutDirectory = input(prompt=">>>")

Output Folder:  C:\Users\m0081459\Documents\Jupyter\Results\Pruebas Motilla\Log Christian\CSV y Script - Jupyter


### 2.2.1 Set and check Output Folder

In [6]:
try:
    if (OutDirectory == None):
        print('Warning','Directory not found. By default, files will be placed in Documents/_Validation/Reports')
        OutDirectory = os.path.expanduser('~/Documents')
    print('Output Directory:' + OutDirectory)
except:
    print('Error','Select a valid directory to store output files')

Output Directory:C:\Users\m0081459\Documents\Jupyter\Results\Pruebas Motilla\Log Christian\CSV y Script - Jupyter


In [7]:
# Sort files by date
import_extension = 'csv'
files = [i for i in sorted(glob.glob('*.{}'.format(import_extension)), key=os.path.getmtime)]

## 2.3 Define Test Name

In [8]:
print("Introduce Test Name")
test = input(prompt=">>>")

Introduce Test Name:  CL10


## 2.3 Define Project

In [9]:
project = request_project(project_question)

Introduce Project
1 :  STEP1
2 :  BEV
3 :  PHEV


>>> 1


STEP1  selected.


## 2.4 Create Output Subfolders

In [10]:
dirName = OutDirectory + '\Combined csvs'
dirPlot = OutDirectory + '\Plots'

try:
    os.mkdir(dirName)
    print('Directory ' , dirName, ' Created ') 
except FileExistsError:
    print('Directory ' , dirName, ' already exists')
    
try:
    os.mkdir(dirPlot)
    print('Directory ' , dirPlot, ' Created ') 
except FileExistsError:
    print('Directory ' , dirPlot, ' already exists')

Directory  C:\Users\m0081459\Documents\Jupyter\Results\Pruebas Motilla\Log Christian\CSV y Script - Jupyter\Combined csvs  already exists
Directory  C:\Users\m0081459\Documents\Jupyter\Results\Pruebas Motilla\Log Christian\CSV y Script - Jupyter\Plots  already exists


## 2.5 Load and combine csv Files

### 2.5.1 Requests Simplification rate

In [11]:
#Request simplification rate
prompt = ">>>"
print("Introduce Simplification rate: ")
try:
    simplification_rate = int(input(prompt=prompt))
except ValueError:
    print("Invalid number. Simplification rate set to 10.")
    simplification_rate = 10

Introduce Simplification rate: 


>>> 20


### 2.5.2 Import CSV

In [12]:
df = pd.concat([pd.read_csv(f, sep=';', low_memory=False, thousands='.', decimal=',',error_bad_lines=False, lineterminator='\n') for f in files])

#Clean up the file
df = df[pd.notnull(df['Time'])] #Remove rows where the column Time contains NaN values

#If the sampling rate is to high use the simplifacation rate variable to reduce the number of data points to plot
df = df.iloc[np.arange(0,df.shape[0], simplification_rate)] #Filters only 1 out of X rows to speed up the ploting process
df['Measurement'] = np.arange(0,df.shape[0])
df = df.reset_index(drop=True)

# 3. DataFrame Processing

## 3.1 Prepare dataframe

In [13]:
#Prepare signals
if 'OBC_Fault' in df: 
    df['OBC_Fault'] = df['OBC_Fault'].map({0:0, 64:1, 128:2, 255:3}) #Map signal OBC Fault
if 'OBC_PowerMax' in df: 
    df['OBC_PowerMax'] = df['OBC_PowerMax']/1000; #Convert to kW

#Create a new column that shows total test time
df['Test_time'] = df['Measurement'] * simplification_rate / 3600 #Time from the beginning of the test in hours

#Create a column with datatime format
data_time = df['Date'] + " " + df['Time'];
pd.to_datetime(data_time, errors='coerce');
df['Date_Time'] = data_time

## 3.2 Define columns to plot

In [14]:
#MAIN COLUMNS
Main_labels = main_columns(df)

In [15]:
#TEMPERATURE 
Temp_labels = temp_columns(df, project[1])

In [16]:
#MEASUREMENTS FROM HV AC I/O
HVAC_labels_V = hvac_v_columns(df, project[1])
HVAC_labels_I = hvac_i_columns(df, project[1])
HVAC_labels_P = hvac_p_columns(df, project[1])

HVAC_labels = merge_dicts(HVAC_labels_V, HVAC_labels_I,HVAC_labels_P)

In [17]:
#MEASUREMENTS FROM HV DC I/O
HVDC_labels_V = hvdc_v_columns(df, project[1])
HVDC_labels_I = hvdc_i_columns(df, project[1])
HVDC_labels_P = hvdc_p_columns(df, project[1])

HVDC_labels = merge_dicts(HVDC_labels_V, HVDC_labels_I,HVDC_labels_P)

In [18]:
#MEASUREMENTS FROM LV DC I/O
LVDC_labels_V = lvdc_v_columns(df, project[1])
LVDC_labels_I = lvdc_i_columns(df, project[1])
LVDC_labels_P = lvdc_p_columns(df, project[1])

LVDC_labels = merge_dicts(LVDC_labels_V, LVDC_labels_I, LVDC_labels_P)

In [19]:
#STATUS
Status_labels = status_columns(df, project[1])

In [20]:
#DATA LOGGER
DL1_labels = dl1_columns(df)
DL2_labels = dl2_columns(df)

## 3.3 Combine labels

In [21]:
labels = merge_dicts(Main_labels, Temp_labels, HVAC_labels, HVDC_labels, LVDC_labels, Status_labels, DL1_labels, DL2_labels)
numeric_labels = merge_dicts(Temp_labels, HVAC_labels, HVDC_labels, LVDC_labels, Status_labels, DL1_labels, DL2_labels)
columns = get_keys(labels)

## 3.4 DataFrame to plot
Create an auxiliar dataframe from the original df with only the columns to plot

In [22]:
#Remove all rows if theres an Nan in any of the columns to plot
df = df[df[get_keys(labels)].notna()]
df = df[df[get_keys(labels)] != "Not read"]

#With some files commas are not properly imported. The following conversion replaces commas by dots and convert all rows that should be numeric to float.
df[get_keys(numeric_labels)] = df[get_keys(numeric_labels)].replace('\.','', regex=True).replace(',','.', regex=True).astype(float)

#We create a copy of the dataframe filtering the columns included in the labels dictionaries
df_plot = df[columns].copy()

#Change Columns names to its labels
df_plot.rename(columns=labels,inplace=True)

## 3.5 Creates and exports a new simplified CSV

In [23]:
#Generate a csv file combining all selected files and with the simplification rate applied
export_extension = 'csv'
try:
    df.to_csv(dirName + '\\' + test + ' '  + 'Combined CSV' + '.' + export_extension,sep=';',decimal=',',index=False)
    df_plot.to_csv(dirName + '\\' +  test + ' '  + 'Combined CSV Simplified' + '.' + export_extension,sep=';',decimal=',',index=False)
    print("Success","File creation completed")
except:
    print("Error","File creation failed")

Success File creation completed


# 4. Plot measurements

## 4.0 Set image size and format

In [24]:
file_type = ".html"
os.chdir(dirPlot)
resolution = [1920, 1080]
resolution_4k = [4096, 2160] 

## 4.1 Temperature

In [25]:
#SHOW TEMPERATURES
fig_temp = plot_1_yaxis("Temperature Measurements", df_plot, "Test time", get_labels(Temp_labels), "Temperature[C]", [-40,100])

## 4.2 HV AC Measurements

In [26]:
#SHOW HV AC Measurements
fig_ac = plot_2_yaxes("HV AC Measurements", df_plot, "Test time", get_labels(HVAC_labels_V), get_labels(HVAC_labels_I), "Voltage[V]", "Current[A]", [0,700], [0,50])

## 4.3 HV DC Measurements

In [27]:
#SHOW HV DC Measurements
fig_hv = plot_2_yaxes("HV DC Measurements", df_plot, "Test time", get_labels(HVDC_labels_V), get_labels(HVDC_labels_I), "Voltage[V]", "Current[A]", [0,500], [0,50])

## 4.4 LV DC Measurements

In [28]:
#SHOW LV DC Measurements
fig_lv = plot_2_yaxes("LV DC Measurements", df_plot, "Test time", get_labels(LVDC_labels_V), get_labels(LVDC_labels_I), "Voltage[V]", "Current[A]", [0,20], [0,250])

## 4.5 Status

In [29]:
#SHOW Status Measurements
max_y = np.max(df_plot[get_labels(Status_labels)].max())
fig_status = plot_1_yaxis("Status", df_plot, "Test time", get_labels(Status_labels), "Status", [0,max_y])

## 4.6 Temperature, HV AC and HV DC

In [30]:
#Plot Temperature HV AC and HV DC
plot_figs("Temperature vs HVAC vs HVDC", fig_temp, fig_ac, fig_hv)

## 4.7 HV AC, HV DC, LV DC

In [31]:
#Plot Temperature HV AC and HV DC
plot_figs("HVAC vs HVDC vs LVDC", fig_ac, fig_hv, fig_lv)

## 4.7 Temperature, HV AC, HV DC and LV DC

In [32]:
#Plot Temperature HV AC, HV DC and LV DC
plot_figs("Temperature vs HVAC vs HVDC vs LVDC", fig_temp, fig_ac, fig_hv, fig_lv)

## 4.8 Temperature, HV AC, HV DC and Status

In [33]:
#Plot Temperature HV AC, HV DC and Status
plot_figs("Temperature vs HVAC vs HVDC vs Status", fig_temp, fig_ac, fig_hv, fig_status)

# 5. Create Python script

In [34]:
#os.chdir(r'C:\Users\m0081459\Documents\Jupyter')
#!jupyter nbconvert --to script Import_And_Plot_csv_v2.ipynb

# __TO DO__
Importar las señales desde el excel
1. _Crear varias páginas_
2. _En cada página poner una columna que sea "Column" y "Alias"_
3. _Importar como dictionary las dos columnas_
4. _Pintar una gráfica por cada sheet del excel_
5. _Buscar las columnas en el df_
6. _Incluir unas columnas comunes para cada plot y luego las especificadas_
7. _Step1 o Step2 selector?_
